In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torchvision.ops import nms

In [ ]:
# yolo = cv2.dnn.readNet('../input/yolo-v3-weights/yolov3.weights', '../input/file123/darknet/darknet/darknet-master/cfg/yolov3.cfg')
# layer_names = yolo.getLayerNames()
# output_layers = [layer_names[x[0] - 1] for x in yolo.getUnconnectedOutLayers()]

In [ ]:
# def detect_face(image):
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#     plt.imshow(image)
#     grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(grayscale_image, 1.2, 7)
#     for (x,y,w,h) in faces:
#         img = cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),5)

In [ ]:
# def my_yolo(image):
#     yolo.setInput(cv2.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0), swapRB = True, crop = False))
#     out = yolo.forward(output_layers)
#     confidences = []
#     boxes = []
#     width = image.shape[1]
#     height = image.shape[0]
#     for curr_out in out:
#         for detection in curr_out:
#             scores = detection[5:]
#             class_id = np.argmax(scores)
#             confidence = scores[class_id]
#             if confidence > 0.1 and class_id == 0:
#                 center_x = int(detection[0] * width)
#                 center_y = int(detection[1] * height)
#                 w = int(detection[2] * width)
#                 h = int(detection[3] * height)
#                 x = center_x - w / 2
#                 y = center_y - h / 2
#                 confidences.append(float(confidence))
#                 boxes.append([x, y, x + w, y + h])
#     boxes = torch.tensor(boxes)
#     confidences = torch.tensor(confidences)
#     iou_threshold = 0.2
#     indices = nms(boxes, confidences, iou_threshold)
#     for i in indices:
#         cv2.rectangle(image, (torch.round(boxes[i][0]), torch.round(boxes[i][1])), (torch.round(boxes[i][2]), torch.round(boxes[i][3])), (0, 0, 0), 2)
#         cv2.putText(image, 'person', (torch.round(boxes[i][0]) - 10, torch.round(boxes[i][1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
#     plt.imshow(image)
#     boxes = [boxes[i] for i in indices]
#     return boxes

In [ ]:
# image = plt.imread('../input/vr-project1-dataset/image1.jpg')
# boxes = my_yolo(image)

In [ ]:
# for i in range(len(boxes)):
#     curr_img = cv2.imread('../input/vr-project1-dataset/image1.jpg')
#     curr_img = curr_img[int(boxes[i][1]) : int(boxes[i][3]), int(boxes[i][0]) : int(boxes[i][2])]
#     detect_face(curr_img)

In [ ]:
class data_convert(Dataset):
    def __init__(self, data):
        self.dataframe = data
    
    def __getitem__(self, ind):
        return self.dataframe[ind]
    
    def __len__(self):
        return len(self.dataframe)

In [ ]:
data = np.load('../input/vrproj/data.npy')
target = np.load('../input/vrproj/target.npy')

In [ ]:
combined_data = []

for i in range(len(data)):
    
    temp_data = []
    fin_data = []
    for j in range(100):
        curr_data = []
        for k in range(100):
            curr_data.append(data[i][j][k][0])
        temp_data.append(curr_data)
    fin_data.append(temp_data)
    
    if target[i][0] == 0:
        combined_data.append((fin_data, 1))
    else:
        combined_data.append((fin_data, 0))

In [ ]:
def batch_generator(curr_batch):
    images = []
    labels = []
    for x in curr_batch:
        images.append(x[0])
        labels.append(x[1])
    images = torch.tensor(images, dtype = torch.float32)
    labels = torch.tensor(labels, dtype = torch.long)
    return images, labels

In [ ]:
data = data_convert(combined_data)
data_train, data_test = train_test_split(data, test_size = 0.1, shuffle = True)
data_train = torch.utils.data.DataLoader(data_train, batch_size = 32, shuffle = True, collate_fn = batch_generator)
data_test = torch.utils.data.DataLoader(data_test, batch_size = 32, shuffle = True, collate_fn = batch_generator)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 200, 3, 1, 0)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(200, 100, 3, 1, 0)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(23 * 23 * 100, 50)
        self.fc2 = nn.Linear(50, 2)
        self.soft = nn.Softmax(dim=1)
        self.drop = nn.Dropout()

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 23 * 23 * 100)
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.soft(self.fc2(x))
        return x

net = Net()

In [ ]:
my_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(my_device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001,momentum=0.9)
criterion.to(my_device)

In [ ]:
validation_losses_test = []
validation_losses_train = []

for epoch in range(100): 
    running_loss_test = 0.0
    running_loss_train = 0.0
    for i, data in enumerate(data_train, 0):
        inputs, labels = data
        inputs, labels = inputs.to(my_device), labels.to(my_device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
    validation_losses_train.append((running_loss_train * 32)/len(data_train))
    
    with torch.no_grad():
        for data in data_test:
            inputs,labels = inputs.to(my_device), labels.to(my_device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            running_loss_test += loss.item()
        validation_losses_test.append((running_loss_test * 32)/len(data_test))

In [ ]:
plt.plot(validation_losses_train, label = 'validation_train')
plt.legend()

In [ ]:
plt.plot(validation_losses_test, label = 'validation_test')
plt.legend()

In [ ]:
correct = 0.0
total = 0.0
with torch.no_grad():
    for data in data_test:
        inputs, labels = inputs.to(my_device), labels.to(my_device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100.0 * correct / total))

In [ ]:
PATH = './mask_classifier.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))